In [1]:
from playwright.async_api import async_playwright
import requests
from random import randint

SCRAPEOPS_API_KEY = 'c36424ca-4337-481c-85f9-43696441d7ea'

def get_headers_list():
  response = requests.get('http://headers.scrapeops.io/v1/browser-headers?api_key=' + SCRAPEOPS_API_KEY)
  json_response = response.json()
  return json_response.get('result', [])

def get_random_header(header_list):
  random_index = randint(0, len(header_list) - 1)
  return header_list[random_index]

header_list = get_headers_list()

In [2]:

pw = await async_playwright().start()

browser = await pw.chromium.launch(headless=False)
page = await browser.new_page()

# note all methods are async (use the "await" keyword)
# await page.goto("http://scrapfly.io/")


In [6]:
user_agent = get_random_header(header_list)['user-agent']

context = await browser.new_context(
                    extra_http_headers = {'user-agent': user_agent}
    )
page = await context.new_page()

In [25]:
await page.goto('https://www.budget.com/en/home')

<Response url='https://www.budget.com/en/home' request=<Request url='https://www.budget.com/en/home' method='GET'>>

In [8]:
await page.click('button[data-click=close]')

In [9]:
await page.click('#usRedirect')

In [26]:
page.fill('#PicLoc_value','Miami Intl Airport, MIA')

<coroutine object Page.fill at 0x7fb430382640>

In [22]:
await page.click('#PicLoc_value')
for letra in 'miami':
    await page.keyboard.press(letra)

In [23]:
await page.click('div[class=angucomplete-results]')

In [11]:
div[class=angucomplete-results]

In [12]:
await page.keyboard.press('Enter')

In [ ]:

# to stop browser on notebook close we can add a shutdown hook:
def shutdown_playwright():
    await browser.close()
    await pw.stop()

In [ ]:
from playwright.sync_api import sync_playwright
from playwright_stealth import stealth_sync
from time import sleep
import datetime
from bs4 import BeautifulSoup
import json




# username = 'j9f5awr7br'
# password = '11rn5hwz9s'
# server = 'http://resi.proxyscrape.com:8000'
server = 'zproxy.lum-superproxy.io:22225'
username = 'brd-customer-hl_4ba48506-zone-isp'
password = '5flzp7i4gt23'

proxies = {
		'server': server,
		'username': username,
		'password': password
	}



retirada = datetime.datetime(2023,6,10)
devolucao = datetime.datetime(2023,6,12)
hora_retirada = '10:00'

def save_json (response, results):
    try:
        results.append(
            {
            'url': response.url,
            'data': response.json(),
            }
        )
    except:
        pass

def route_intercept(route):
    if route.request.resource_type == "image":
        print(f"Blocking the image request to: {route.request.url}")
        return route.abort()
    if "localiza.com" not in route.request.url:
        print(f"blocking {route.request.url} as it contains Google")
        return route.abort()
    return route.continue_()

def texto_tag(page,tag):
    page.is_visible(tag)
    html = page.inner_html(tag)
    soup = BeautifulSoup(html, 'html.parser')

    return soup.text


def busca_data (page, data, indice):
    mes_ext = ' ' + meses.get(data.month) + f' DE {data.year} '

    page.is_visible(f'mat-datepicker-{indice}')
    
    while texto_tag(page,'span[class=mat-button-wrapper]') != mes_ext:
        page.is_visible('button[aria-label="Next month"]')
        page.click('button[aria-label="Next month"]')

    page.is_visible('div[class=mat-calendar-body-cell-content]')
    page.get_by_text(f' {data.day} ', exact=True).click()

def busca_hora (page, hora):
    page.is_visible('span[class=mat-option-text]')
    page.get_by_text(f' {hora} ', exact=True).click()      

def gera_ran(min=4000, max =7000):
    return randint(min,max)/1000

def run (playwright):
    browser = playwright.chromium.launch(headless=False,slow_mo=50
                                         #,   proxy = proxies
                                         )

    user_agent = get_random_header(header_list)['user-agent']

    context = browser.new_context(
                        extra_http_headers = {'user-agent': user_agent}
        )
    page = context.new_page()
    stealth_sync(page)
    #page.route("**/*", route_intercept)

    page.goto('https://www.budget.com/en/home', timeout = 100000)

    sleep(gera_ran())
    page.click('input#mat-input-1')
    for letra in 'guarulhos':
        page.keyboard.press(letra)
    sleep(gera_ran())
    page.click('span[class=places-list__item__name]')
    print('loja selecionada')
    
    busca_data (page, retirada,0)
    sleep(gera_ran())

    busca_hora (page, ' 10:00 ')
    sleep(gera_ran())
    print('data selecionada')
    busca_data (page, devolucao,1)
    sleep(gera_ran())

    page.is_visible('span[class=ng-star-inserted]')
    page.click('span[class=ng-star-inserted]')
    print('data selecionada')
    sleep(gera_ran())
    print('aguardando resposta final')

    page.is_visible('#main-content')

    
    results = []
    sleep(20)
    page.wait_for_url('https://www.localiza.com/brasil/pt-br/reservas/passo-2')
    page.on('response', lambda response: save_json(response, results))

    return results

    


with sync_playwright() as playwright:
    data = run(playwright)
    print(data)
    hora_pesquisa = int(datetime.datetime.timestamp(datetime.datetime.now()))
    with open(rf'json/resultado_{hora_pesquisa}.json', 'w') as f:
        json.dump(data, f)